In [1]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [2]:
import os
from datetime import datetime, date, timedelta
import pandas as pd 
import numpy as np
import functions as f

folder_path = 'D:/Offline Data/Processed Data/Nifty_1_Minute_ICICI'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
sorted_files = sorted(csv_files, key=lambda x: datetime.strptime(x.split('_')[0] + '_' + x.split('_')[1] + '_' + x.split('_')[2], '%Y_%m_%d'))
nn = 0

for i in range(0, len(sorted_files)):
    print(sorted_files[i])

    sym_df = pd.DataFrame()

    file_path = os.path.join(folder_path, sorted_files[i])
    df = pd.read_csv(file_path)

    current_date = str(pd.to_datetime(df['datetime'].iloc[0]).date())
    is_date_found = f.check_date_in_records(current_date)

    if is_date_found == "date_not_found":

        filter_df = df[(df.open.between(200, 300)) & df.symbol.str.endswith('CE')]
        unique_symbols_CE = filter_df['symbol'].unique()

        time_range = pd.date_range(start='09:15:00', end='15:30:00', freq='3T').strftime('%H:%M:%S')
        symbol_store = None
        n = 0

        for i in range(len(time_range)):
            # print(time_range[i])

            df_strike_price = f.add_strike_prices_in_new_column(time_range[i], unique_symbols_CE, df)
            CE_strike_df = f.find_nearest_greater(df_strike_price)

            sym_df = pd.concat([sym_df, CE_strike_df], ignore_index=True)

            if CE_strike_df['symbol'].iloc[0] != symbol_store:
                symbol_store = CE_strike_df['symbol'].iloc[0]
                n += 1
                nn += 1

            # ----------------------------------------

        print(f'Symbol chaged: {n} times')

        current_date = str(pd.to_datetime(df['datetime'].iloc[0]).date())
        file_name = f'{current_date}.csv'
        output_path = os.path.join('D:\Offline Data\Processed Data\Get_symbol', file_name)
        sym_df.to_csv(output_path, index=False)

        f.update_date_records(current_date, str(datetime.now().date()))

print(f'Average symbol change: {nn/len(sorted_files)} times')

ValueError: unconverted data remains: .csv

In [ ]:
sym_df